In [1]:
import matplotlib.pyplot as plt
#from PIL import Image
import matplotlib.image as mpimg
import numpy as np
#from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras import layers, models
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply
from tensorflow.keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import tensorflow.metrics
import pandas as pd
from sklearn.model_selection import train_test_split
import os, shutil, random
from sklearn.metrics import confusion_matrix
import itertools
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler

In [3]:
combined_a = tensorflow.keras.models.load_model('model.h5', compile=False)

### Define preprocessing function for demographic data

In [32]:
def preprocess_dem(train,test):


    #replace missing age values by median
    train['age'] = train['age'].fillna(train['age'].median())
    test['age'] = test['age'].fillna(train['age'].median())
    
    # performin min-max scaling each continuous feature column to
    # the range [0, 1]
    cs = MinMaxScaler()
    train["age"] = cs.fit_transform(train["age"].values.reshape(-1,1))
    test["age"] = cs.transform(test["age"].values.reshape(-1,1))
 
    train['sex'].replace("unknown",train['sex'].value_counts().index[0], inplace=True)
    train["sex"] = np.where(train['sex']=="female",1,0)
    
    test['sex'].replace("unknown",train['sex'].value_counts().index[0], inplace=True)
    test["sex"] = np.where(test['sex']=="female",1,0)
    
    # return the concatenated training and testing data
    return (train, test)

### Load demographic data for all images (inkl. augmented images)

In [6]:
diagnose = ['bkl', 'nv', 'df', 'mel', 'vasc', 'bcc', 'akiec']

train_dir = "/Users/leona/PythonWork/Github_Projects/Final_Pro/data/ISIC2018_Task3_Training_Input/train/"
validation_dir = "/Users/leona/PythonWork/Github_Projects/Final_Pro/data/ISIC2018_Task3_Training_Input/validation/"

In [14]:
def get_imagelist(classes,path):
    img_list = []
    for c in classes:
        img_list.append(os.listdir(path + c))
    img_list = [item for sublist in img_list for item in sublist]
    return img_list

In [15]:
train_list = get_imagelist(diagnose,train_dir)
val_list = get_imagelist(diagnose,validation_dir)

#### Get main dataframe

In [19]:
meta = pd.read_pickle("./meta.pkl")

In [21]:
train = pd.read_pickle("./aug_train.pkl")

In [22]:
validation = pd.read_pickle("aug_test.pkl")

In [33]:
train_pr, validation_pr = preprocess_dem(train,validation)

/Users/leona/anaconda3/envs/finalpro/lib/python3.6/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


### Define custom image generator

In [ ]:
def createGenerator(df):

    while True:
        # suffled indices    
        idx = df.sample(n=10, replace=False).index
        # create image generator
        datagen = ImageDataGenerator(
            preprocessing_function= tensorflow.keras.applications.mobilenet.preprocess_input)

        batches = datagen.flow_from_dataframe(df.loc[idx], x_col="full_id",y_col="dx", shuffle=False)
        for batch in batches:
        
            yield [batch[0], df.loc[idx[["age", "sex"]].values]], batch[1]